# Annotate LTR retrotransposons using exisiting consensus sequences

#### Example data: annotate RLC_BdisC022 on chromosome 5 of *B. distachyon*

In [12]:
#%% Define paths to input files, set parameters

# path to genome
annotendum = 'data/Bdistachyon_314_v3.0.fa'
# path to TE consensus sequences
consensus_seqs = 'data/RLC_BdisC022.fasta'

# only retain blast hits which cover at least min_ltr_cov of the LTRs
min_ltr_cov = 0.80
#
imprec = 0.2
write_files = True

In [13]:
#%% Some classes and functions used for the annotation
import os
import subprocess

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


#%% some functions and classes

class LTR_hit:
    
    def __init__(self, hit):
        
        coords = map(int, hit[7:9])
        strt, end = min(coords), max(coords)
        
        self.hit = {'aln_len' : int(hit[0]),
                    'pid' : float(hit[1]),
                    'eval' : float(hit[10]),
                    
                    'TE' : hit[2].split('.')[0],
                    'TE_strt' : int(hit[3]),
                    'TE_end' : int(hit[4]),
                    
                    'contig' : hit[5],
                    'c_strnd' : hit[6],
                    'c_strt' : strt,
                    'c_end' : end,
                    'c_seq' : Seq(hit[9])
                    }


    def add_flanking(self, contigs, n=8):
        
        chrmsm = self.hit['contig']
        strt = self.hit['c_strt']
        end = self.hit['c_end']
        
        self.five_prime = contigs[chrmsm][strt-(n+1) : strt-1]
        
        self.three_prime = contigs[chrmsm][end : end+n]
        
        if self.hit['c_strnd'] == 'minus':
            self.five_prime = self.five_prime.reverse_complement()
            self.three_prime = self.three_prime.reverse_complement()
            
        
def blastn(query, subject, outfile=False):
    
    cmd = ['blastn',
           '-query', query,
           '-subject', subject,
           '-outfmt', '6 length pident qseqid qstart qend sseqid sstrand sstart send sseq evalue'
           ]

    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    output = proc.stdout.read()
    blast_out = output.splitlines()
    
    if outfile:
        with open(outfile, 'w') as f:
            for line in blast_out:
                f.write(line + '\n')

    return [line.split('\t') for line in output.splitlines()]


def is_tsd(five_seq, three_seq, overlap=4):

    tsd_max_len = len(five_seq)
    
    four_mers_l = []
    four_mers_r = []
    
    if five_seq and three_seq:
     
        for i in xrange(tsd_max_len):
            if i+4 < tsd_max_len+1:
                four_mers_l.append(five_seq[i:i+4])
                four_mers_r.append(three_seq[i:i+4])
                
    if any(x in four_mers_r for x in four_mers_l):
        return '1'
    else:
        return '0'

    
def check_LTR_context(ltr_h, contigs, te_info, n=500, seq_cov=0.1):
    """ Parameters:
        - n: number of TE internal base pairs considered
        - seq_cov: how much of these base pairs must be covered?
    """
    
    chrmsm = ltr_h.hit['contig']    
    
    coords = [ltr_h.hit['c_strt'], ltr_h.hit['c_end']]
    strt, end = min(coords), max(coords)
    te = ltr_h.hit['TE']
    
    # attenzione, strand confusioni!
    if ltr_h.hit['c_strnd'] == 'plus':
        left_seq = contigs[chrmsm][strt-n:strt]
        right_seq = contigs[chrmsm][end:end+n]
    else:
        left_seq = contigs[chrmsm][end:end+n].reverse_complement()
        right_seq = contigs[chrmsm][strt-n:strt].reverse_complement()
    
    probe_left = SeqRecord(left_seq, id='probe_left', name='probe_left', description='')
    probe_right = SeqRecord(right_seq, id='probe_right', name='probe_right', description='')

    # regions expected to be covered on consensus
    right_exp = (te_info[te]['ltr_len'], te_info[te]['ltr_len'] + n)
    left_exp = (te_info[te]['full_len'] - te_info[te]['ltr_len'] - n,  te_info[te]['full_len'] - te_info[te]['ltr_len'])

    SeqIO.write([probe_left, probe_right], 'tmp.fasta', 'fasta')
    blastout = blastn('tmp.fasta', te + '/' + te + '.fasta')
    os.remove('tmp.fasta')
    
    best = 0

    if blastout:
        for hit in blastout:
                  
            strt = min(map(int, hit[7:9]))
            end = max(map(int, hit[7:9]))
            
            if hit[2] == 'probe_left':
                ol = is_overlapping(left_exp, (strt, end))               
                if not ol:
                    out = 'solo_LTR'
                elif ol > best:
                    best = ol
                    out = 'three_prime_LTR' if ol > seq_cov else 'NA'
                    
    
            elif hit[2] == 'probe_right':
                ol = is_overlapping(right_exp, (strt, end))
                if not ol:
                    out = 'solo_LTR'
                elif ol > best:
                    best = ol
                    out = 'five_prime_LTR' if ol > seq_cov else 'NA'
    else:
        out = 'solo_LTR'
                    
    return out
            
            
def is_overlapping(a,b):
    a_range = set(xrange(a[0], a[1]))
    b_range = set(xrange(b[0], b[1]))

    intersection = a_range & b_range
    union = a_range | b_range

    if len(intersection) == 0:
        return False
    else:
        return len(intersection) / float(len(union))

    
def assign_TE_cat(a, b, te_info, imprec, min_ltr_cov=0.8):
    
    """ Determine TE insertion category:
    
    - full length
    - solo LTR
    - single LTR
    - tsd yes or no
    
    1 in the output means TSD is present
    """
    
    te = a.hit['TE']
    if a.hit['aln_len'] < len(ltrs[te]) * min_ltr_cov:
        return 'trunc', 0, 0
    
    aa_tsd = is_tsd(a.five_prime, a.three_prime)      
    a_cntxt = check_LTR_context(a, contigs, te_info)
    
    if not b or te != b.hit['TE']: 
        return 'single', aa_tsd, a_cntxt
      
    else:
        b_cntxt = check_LTR_context(b, contigs, te_info)
        internal = te_info[te]['full_len'] -(2*te_info[te]['ltr_len'])
        interval = b.hit['c_strt'] - a.hit['c_end']
        ab_tsd = is_tsd(a.five_prime, b.three_prime)
        cntxt = a_cntxt + '-' + b_cntxt
        
        if (1-imprec)*internal < interval < (1+imprec)*internal and a.hit['c_strnd'] == b.hit['c_strnd']:
            return 'paired', ab_tsd, cntxt
        
        else:
            return  'single', aa_tsd, a_cntxt

In [14]:
#%% Load sequence data and get the LTR sequence for each element
contigs = {seq_record.id: seq_record.seq for seq_record in SeqIO.parse(annotendum, "fasta")}

tes = {seq_record.id: seq_record for seq_record in SeqIO.parse(consensus_seqs, "fasta")}
basename = consensus_seqs.split('/')[-1]
ltrs = {}

os.mkdir('tmp')
os.chdir('tmp')

for seq in tes:
    
    SeqIO.write(tes[seq], seq + '.fasta', 'fasta')
    
    selfblast = blastn(seq + '.fasta', seq + '.fasta')[1:]
    selfblast.sort(key=lambda x: (int(x[7])))  
    
    ltrs[seq] = SeqRecord(Seq(selfblast[0][-2]), id=seq, name=seq, description='')      
    os.remove(seq + '.fasta')
    
os.chdir('..')
os.rmdir('tmp')


In [15]:
#%% Blast LTRs and parse hits

blast_d = {}
te_info = {}
overlaps = []


for te in tes:
    
    """ Blast LTR of each TE family against the assembly, 
    create LTR_hit object and extract sequences flanking the hit
    """
    
    print 'Annotating ' + te

    if te in ignore:
        continue
    
    os.mkdir(te)

    SeqIO.write(tes[te], te + '/' + te + '.fasta', 'fasta')
    SeqIO.write(ltrs[te], te + '/' + te + '.LTR.fasta', 'fasta')
    full_len = len(tes[te])
    ltr_len = len(ltrs[te])    
    te_info[te] = {'full_len' : full_len, 'ltr_len' : ltr_len}


    """ Blast LTR against targets, create dictionary with blast output with chromosomes as keys
    """
    stats[te] = {'retained' : 0 , 'trunc' : 0, 'overlap' : 0}
    blast_out = blastn(te + '/' + te + '.LTR.fasta', annotendum) 
    
    for hit in blast_out:
        
        # Only consider hits covering at least 80% of the LTR
        aln_len = int(hit[0])
#        if aln_len < min_ltr_cov*ltr_len:
#            stats[te]['trunc'] +=1
#            continue
                    
        chrmsm = hit[5]
        if chrmsm not in blast_d:
            blast_d[chrmsm] = {}
    
        coords = map(int, hit[7:9])
        strt, end = min(coords), max(coords)
        
        ltr_h = LTR_hit(hit)
        ltr_h.add_flanking(contigs)
        
        
        # overlapping coordinates: keep hit with higher PID
        overlap = False
        
        worse_hits_to_remove = []
        for region in blast_d[chrmsm]:
            ol = is_overlapping((strt,end), region)
            
            if ol: 
                overlap = True
                if ltr_h.hit['eval'] < blast_d[chrmsm][region].hit['eval']:
                    worse_hits_to_remove.append(region)
        
        if worse_hits_to_remove:
            for h in worse_hits_to_remove:
                blast_d[chrmsm].pop(h)
            blast_d[chrmsm][(strt, end)] = ltr_h
                
        elif overlap == False:
            blast_d[chrmsm][(strt, end)] = ltr_h


Annotating RLC_BdisC022


In [ ]:
#%% Identify full length and solo copies, write gff and fasta files
annot = []
seqs = {}
ltr_seqs = {}
annot_stats = {}

for chrmsm in blast_d:
        
        """ Order blast hits and compare adjacent hits in order
        to distinguish solo from full length elements,
        and to identify overlapping annotations
        """
        
        coords = blast_d[chrmsm].keys()
        coords.sort(key=lambda x: (int(x[0])))

        a = False
        
        for i,x  in enumerate(coords):
            
            if not a: # initiate contig
                a = blast_d[chrmsm][x]
                
                if len(coords) == 1: # single hit in the contig
                    b = False
                    cat, tsd, ltr_cntxt = assign_TE_cat(a, b, te_info, imprec)
                    
                else:
                    continue
            
            b = blast_d[chrmsm][x]
            cat, tsd, ltr_cntxt = assign_TE_cat(a, b, te_info, imprec)
            
        
            """ Put together output info
            """
            TE = a.hit['TE']
            strt = a.hit['c_strt']
            flanking_left = a.five_prime
            # careful with fasta header names, avoid - and :
            name = TE + '_' + chrmsm + '_' + str(strt)
            
            if TE not in annot_stats:
                annot_stats[TE] = {'paired' : 0, 'single' : 0, 'solo_LTR' : 0, 'full' : 0, 'other' : 0, 'trunc':0}
                seqs[TE] = []
                ltr_seqs[TE] = []
            
            # strand
            strand = '+' if a.hit['c_strnd'] == 'plus' else '-'
            
            # end coordinate
            if cat == 'paired':
                flanking_right = b.three_prime
                end = b.hit['c_end']
            else:
                flanking_right = a.three_prime
                end = a.hit['c_end']
                

            # Categorize and get LTR sequences 
            if cat == 'single':
                if ltr_cntxt == 'solo_LTR'and tsd == '1':
                    subcat = 'solo_LTR'
                else:
                    subcat = 'other'
                    
                ltr_rec = SeqRecord(a.hit['c_seq'], id=name, name=name, description=cat + '_' + subcat)
                ltr_seqs[TE].append(ltr_rec)
                
                
            if cat == 'paired':
                
                if strand == '-' and ltr_cntxt in ['three_prime_LTR-five_prime_LTR', 'three_prime_LTR-NA', 'NA-five_prime_LTR'] or \
                   strand == '+' and ltr_cntxt in ['five_prime_LTR-three_prime_LTR', 'NA-three_prime_LTR', 'five_prime_LTR-NA']:
                       
                    subcat = 'full' if tsd == '1' else 'other'
                    
                else:
                    subcat = 'other'
    
                ltr_recA = SeqRecord(a.hit['c_seq'], id=name + '_A', name=name + '_A', description=cat + '_' + subcat)
                ltr_recB = SeqRecord(b.hit['c_seq'], id=name + '_B', name=name + '_B', description=cat + '_' + subcat)
                ltr_seqs[TE] += [ltr_recA, ltr_recB]
                
            if cat == 'trunc':
                subcat = 'trunc'
                
                
            # TE sequence and length
            seq = contigs[chrmsm][strt-1 : end]
            aln_len = len(seq)
            if strand == '-':
                seq = seq.reverse_complement()
            rec = SeqRecord(seq, id=name, name=name, description=cat+ '_' + subcat)
            seqs[TE].append(rec)
                

            """ Write reference hits to gff
            """
            info = 'Name=%s,fam=%s,cat=%s,subcat=%s,tsd=%s,ltr_context=%s,five_flanking=%s,three_flanking=%s,aln_len=%s' % \
                    (name, TE, cat, subcat, tsd, ltr_cntxt, flanking_left, flanking_right, aln_len)
                    
            annot.append([chrmsm, 'detettore', 'LTR-RT', str(strt), str(end), '.', strand, '.', info])

            annot_stats[TE][cat] += 1 
            annot_stats[TE][subcat] += 1
            
            # Skip next entry if it was part of a full element
            if cat  == 'paired':
                a = False
            else:
                a = b

In [ ]:
# Write files
                
stats_header = ['TE', 'paired', 'single', 'full', 'solo_LTR', 'other']
              
if write_files:
    annot.sort(key=lambda x: (x[0], int(x[3])))
    with open('LTR-RTs.dez2018.gff', 'w') as out:
        for a in annot:
            out.write('\t'.join(a)+'\n')
    
    for te in seqs:
        SeqIO.write(seqs[te], te + '/' + te + '.all_seqs.fasta', 'fasta')
        SeqIO.write(ltr_seqs[te], te + '/' + te + '.ltr_seqs.fasta', 'fasta')
        
    
    with open('annotation_counts.tsv', 'w') as f:
        f.write('\t'.join(stats_header) + '\n')
        for te in annot_stats:
            outline = [te] + [str(annot_stats[te][c]) for c in stats_header[1:]]
            f.write('\t'.join(outline) + '\n')